<a href="https://colab.research.google.com/github/pvenkatkishn/Healthcare-DataPipleine/blob/main/Redshift_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install vcfpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.2/993.2 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 43.7 MB/s eta 0:00:00
  Created wheel for vcfpy: filename=vcfpy-0.13.8-py2.py3-none-any.whl size=34400 sha256=d22012d6ed7e21252a5495af9bd52b150bb9535f57f97c42fef738d15cac333b
  Stored in directory: /root/.cache/pip/wheels/f8/18/ae/1270a4895ffbb577ccf51ebe1396ce1dc6ce89f851079a2ec2
Successfully built vcfpy


In [ ]:
import vcfpy
import pandas as pd
import os

# Define the directory where your .vcf.gz files are stored
vcf_dir = '/content/drive/My Drive/AWS_Redshift/Genomics_Data/test'
output_dir = '/content/drive/My Drive/AWS_Redshift/Genomics_Data/chr_agg_data.csv'

# Define the function to process and aggregate a single .vcf.gz file
def process_vcf(file_path):
    reader = vcfpy.Reader.from_path(file_path)
    records = []

    for record in reader:
        chrom = record.CHROM
        pos = record.POS
        ref = record.REF
        alt = ','.join(str(a) for a in record.ALT)
        allele_count = record.INFO.get('AC', [0])[0]
        allele_freq = record.INFO.get('AF', [0])[0]

        # Assuming you have metadata for gender and location
        location_name = 'Unknown'  # Replace this with actual extraction if possible
        gender = 'Unknown'  # Replace this with actual extraction if possible

        records.append([chrom, pos, ref, alt, allele_count, allele_freq, location_name, gender])

    # Convert to DataFrame
    df = pd.DataFrame(records, columns=['CHROM', 'POS', 'REF', 'ALT', 'Allele_Count', 'Allele_Frequency', 'Location_Name', 'Gender'])

    # Aggregate by Location_Name and Gender
    aggregated_df = df.groupby(['Location_Name', 'Gender']).agg({
        'Allele_Frequency': 'mean',
        'Allele_Count': 'sum'
    }).reset_index()

    return aggregated_df

# Process each .vcf.gz file and save the output as a .csv file
for file in os.listdir(vcf_dir):
    if file.endswith('.vcf.gz'):
        file_path = os.path.join(vcf_dir, file)
        aggregated_df = process_vcf(file_path)

        # Save each processed file as a CSV
        output_file = os.path.join(output_dir, file.replace('.vcf.gz', '_aggregated.csv'))
        aggregated_df.to_csv(output_file, index=False)
        print(f"Saved aggregated data to {output_file}")


ModuleNotFoundError: No module named 'vcfpy'

In [ ]:
import vcfpy
import pandas as pd

# Load the panel file (assuming it contains columns 'sample', 'pop' for population and 'gender')
panel_file_path = '/content/drive/My Drive/AWS_Redshift/Genomics_Data/integrated_call_samples_v3.20130502.ALL.panel.txt'  # Replace with actual path
panel_df = pd.read_csv(panel_file_path, sep='\t')

# Function to clean the VCF file and include location name and gender
def clean_vcf_with_location(file_path, panel_df, limit=10):
    reader = vcfpy.Reader.from_path(file_path)

    records = []
    count = 0

    for record in reader:
        if count >= limit:
            break

        chrom = record.CHROM
        pos = record.POS
        ref = record.REF

        # Simplify ALT field (keep only the value, remove type information)
        alt = ','.join([str(a.value) for a in record.ALT])

        # Extract relevant fields from the INFO dictionary
        allele_count = record.INFO.get('AC', ['Unknown'])[0]
        allele_freq = record.INFO.get('AF', ['Unknown'])[0]

        # Extract the sample IDs from the calls and map them to the location name and gender
        for call in record.calls:
            sample_id = call.sample
            sample_info = panel_df[panel_df['sample'] == sample_id]

            if not sample_info.empty:
                population_code = sample_info['pop'].values[0]
                gender = sample_info['gender'].values[0]  # Assuming 'gender' is a column in the panel file

                # Map population code to country name
                location_name = country_code_to_name.get(population_code, 'Unknown')

                # Append the relevant data
                records.append([chrom, pos, ref, alt, allele_count, allele_freq, location_name, gender])

        count += 1

    # Convert to a DataFrame for better viewing and manipulation
    df = pd.DataFrame(records, columns=['CHROM', 'POS', 'REF', 'ALT', 'Allele_Count', 'Allele_Frequency', 'Location_Name', 'Gender'])

    return df

# View the cleaned data for the first 10 records
vcf_file_path = '/content/drive/My Drive/AWS_Redshift/Genomics_Data/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz'  # Replace with your actual VCF file path
cleaned_vcf_data = clean_vcf_with_location(vcf_file_path, panel_df, limit=10)
print(cleaned_vcf_data)


      CHROM    POS                     REF ALT  Allele_Count  \
0         1  10177                       A  AC          2130   
1         1  10177                       A  AC          2130   
2         1  10177                       A  AC          2130   
3         1  10177                       A  AC          2130   
4         1  10177                       A  AC          2130   
...     ...    ...                     ...  ..           ...   
25035     1  10616  CCGCCGTTGCAAAGGCGCGCCG   C          4973   
25036     1  10616  CCGCCGTTGCAAAGGCGCGCCG   C          4973   
25037     1  10616  CCGCCGTTGCAAAGGCGCGCCG   C          4973   
25038     1  10616  CCGCCGTTGCAAAGGCGCGCCG   C          4973   
25039     1  10616  CCGCCGTTGCAAAGGCGCGCCG   C          4973   

       Allele_Frequency   Location_Name  Gender  
0              0.425319  United Kingdom    male  
1              0.425319  United Kingdom  female  
2              0.425319  United Kingdom  female  
3              0.425319  United

In [ ]:
# Assuming cleaned_vcf_data is your dataframe with relevant columns
# Group by Gender and Location_Name
grouped_data = cleaned_vcf_data.groupby(['Gender', 'Location_Name']).agg({
    'Allele_Frequency': 'mean',  # Calculate mean allele frequency
    'Allele_Count': 'sum'        # Sum allele count (or any other relevant aggregation)
}).reset_index()

# Display the grouped data
print(grouped_data)


    Gender     Location_Name  Allele_Frequency  Allele_Count
0   female  African American          0.185863        325780
1   female        Bangladesh          0.185863        409552
2   female         Caribbean          0.185863        456092
3   female             China          0.185863       1479972
4   female          Colombia          0.185863        474708
5   female           Finland          0.185863        567788
6   female            Gambia          0.185863        539864
7   female             India          0.185863        837720
8   female             Italy          0.185863        502632
9   female             Japan          0.185863        446784
10  female             Kenya          0.185863        511940
11  female            Mexico          0.185863        297856
12  female           Nigeria          0.185863        949416
13  female          Pakistan          0.185863        446784
14  female              Peru          0.185863        409552
15  female       Puerto 

In [ ]:
import vcfpy
import pandas as pd

# Function to clean the VCF file (excluding the samples list)
def clean_vcf(file_path, limit=10):
    reader = vcfpy.Reader.from_path(file_path)

    records = []
    count = 0

    for record in reader:
        if count >= limit:
            break

        chrom = record.CHROM
        pos = record.POS
        ref = record.REF

        # Simplify ALT field (keep only the value, remove type information)
        alt = ','.join([str(a.value) for a in record.ALT])

        # Extract relevant fields from the INFO dictionary
        allele_count = record.INFO.get('AC', ['Unknown'])[0]
        allele_freq = record.INFO.get('AF', ['Unknown'])[0]

        # Append the necessary fields to the records list
        records.append([chrom, pos, ref, alt, allele_count, allele_freq])
        count += 1

    # Convert to a DataFrame for better viewing and manipulation
    df = pd.DataFrame(records, columns=['CHROM', 'POS', 'REF', 'ALT', 'Allele_Count', 'Allele_Frequency'])

    return df

# View the cleaned data for the first 10 records
vcf_file_path = vcf_file_path = '/content/drive/My Drive/AWS_Redshift/Genomics_Data/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz'  # Replace with your actual VCF file path
cleaned_vcf_data = clean_vcf(vcf_file_path, limit=10)
print(cleaned_vcf_data)


  CHROM    POS                     REF ALT  Allele_Count  Allele_Frequency
0     1  10177                       A  AC          2130          0.425319
1     1  10235                       T  TA             6          0.001198
2     1  10352                       T  TA          2191          0.437500
3     1  10505                       A   T             1          0.000200
4     1  10506                       C   G             1          0.000200
5     1  10511                       G   A             1          0.000200
6     1  10539                       C   A             3          0.000599
7     1  10542                       C   T             1          0.000200
8     1  10579                       C   A             1          0.000200
9     1  10616  CCGCCGTTGCAAAGGCGCGCCG   C          4973          0.993011


In [ ]:
import pandas as pd

# Load the population metadata file
panel_file_path = '/content/drive/My Drive/AWS_Redshift/Genomics_Data/integrated_call_samples_v3.20130502.ALL.panel.txt'
panel_df = pd.read_csv(panel_file_path, sep='\t')

# View the first few rows of the panel file to understand its structure
print(panel_df.head())


    sample  pop super_pop  gender  Unnamed: 4  Unnamed: 5
0  HG00096  GBR       EUR    male         NaN         NaN
1  HG00097  GBR       EUR  female         NaN         NaN
2  HG00099  GBR       EUR  female         NaN         NaN
3  HG00100  GBR       EUR  female         NaN         NaN
4  HG00101  GBR       EUR    male         NaN         NaN


In [ ]:
# Filter the panel_df for a specific population (e.g., GBR for United Kingdom)
gbr_samples = panel_df[panel_df['pop'] == 'GBR']['sample'].tolist()

# Now you can use this list of 'gbr_samples' to filter your VCF data for just UK samples
import pandas as pd

# Read the metadata file
metadata_file_path = '/content/drive/My Drive/AWS_Redshift/Genomics_Data/integrated_call_samples_v3.20130502.ALL.panel.txt'
metadata = pd.read_csv(metadata_file_path, sep='\t')
metadata_cleaned = metadata.drop(columns=['Unnamed: 4', 'Unnamed: 5'])

# Display the cleaned metadata
print(metadata_cleaned.head())

# Check for any remaining issues in the metadata
print(metadata_cleaned.info())



    sample  pop super_pop  gender
0  HG00096  GBR       EUR    male
1  HG00097  GBR       EUR  female
2  HG00099  GBR       EUR  female
3  HG00100  GBR       EUR  female
4  HG00101  GBR       EUR    male
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2504 entries, 0 to 2503
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sample     2504 non-null   object
 1   pop        2504 non-null   object
 2   super_pop  2504 non-null   object
 3   gender     2504 non-null   object
dtypes: object(4)
memory usage: 78.4+ KB
None


In [ ]:
import vcfpy
# Path to the VCF file
vcf_file_path = '/content/drive/My Drive/AWS_Redshift/Genomics_Data/ALL.chr20.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz'

# Open the VCF file using vcfpy
reader = vcfpy.Reader.from_path(vcf_file_path)

# Prepare to collect data
records = []

# Iterate over records in the VCF file
for i, record in enumerate(reader):
    row = {
        'CHROM': record.CHROM,
        'POS': record.POS,
        'ID': record.ID,
        'REF': record.REF,
        'ALT': str(record.ALT[0])
    }
    # Add genotype information for each sample
    for call in record.calls:
        sample_name = call.sample
        genotype = call.data.get('GT', './.')  # Default to './.' if missing
        row[sample_name] = genotype
    records.append(row)

    if i >= 5:  # Limit for testing
        break

# Convert to DataFrame
variants_df = pd.DataFrame(records)

# Display the first few rows
print(variants_df.head())


  CHROM    POS  ID REF                                    ALT HG00096 HG00097  \
0    20  60343  []   G   Substitution(type_='SNV', value='A')     0|0     0|0   
1    20  60419  []   A   Substitution(type_='SNV', value='G')     0|0     0|0   
2    20  60479  []   C   Substitution(type_='SNV', value='T')     0|0     0|0   
3    20  60522  []   T  Substitution(type_='INS', value='TC')     0|0     0|0   
4    20  60568  []   A   Substitution(type_='SNV', value='C')     0|0     0|0   

  HG00099 HG00100 HG00101  ... NA21128 NA21129 NA21130 NA21133 NA21135  \
0     0|0     0|0     0|0  ...     0|0     0|0     0|0     0|0     0|0   
1     0|0     0|0     0|0  ...     0|0     0|0     0|0     0|0     0|0   
2     0|0     0|0     0|0  ...     0|0     0|0     0|0     0|0     0|0   
3     0|0     0|0     0|0  ...     0|0     0|0     0|0     0|0     0|0   
4     0|0     0|0     0|0  ...     0|0     0|0     0|0     0|0     0|0   

  NA21137 NA21141 NA21142 NA21143 NA21144  
0     0|0     0|0     0|

In [ ]:
# Extract sample IDs from VCF DataFrame (assuming you have variants_df from the VCF)
vcf_sample_ids = variants_df.columns[5:]  # Adjust this if your DataFrame structure is different

# Verify matching sample IDs
matching_samples = metadata_cleaned['sample'].isin(vcf_sample_ids)
print(metadata_cleaned[matching_samples].head())


    sample  pop super_pop  gender
0  HG00096  GBR       EUR    male
1  HG00097  GBR       EUR  female
2  HG00099  GBR       EUR  female
3  HG00100  GBR       EUR  female
4  HG00101  GBR       EUR    male


In [ ]:
# Reshape variants_df if necessary to long format for merging (depending on your analysis)
# Example: pivot or melt to align with metadata structure

# Merge datasets
combined_df = pd.melt(variants_df, id_vars=['CHROM', 'POS', 'ID', 'REF', 'ALT'],
                      var_name='sample', value_name='genotype')

# Merge with metadata
integrated_data = combined_df.merge(metadata_cleaned, on='sample')

# Display the integrated dataset
print(integrated_data)


      CHROM    POS  ID REF                                    ALT   sample  \
0        20  60343  []   G   Substitution(type_='SNV', value='A')  HG00096   
1        20  60419  []   A   Substitution(type_='SNV', value='G')  HG00096   
2        20  60479  []   C   Substitution(type_='SNV', value='T')  HG00096   
3        20  60522  []   T  Substitution(type_='INS', value='TC')  HG00096   
4        20  60568  []   A   Substitution(type_='SNV', value='C')  HG00096   
...     ...    ...  ..  ..                                    ...      ...   
15019    20  60419  []   A   Substitution(type_='SNV', value='G')  NA21144   
15020    20  60479  []   C   Substitution(type_='SNV', value='T')  NA21144   
15021    20  60522  []   T  Substitution(type_='INS', value='TC')  NA21144   
15022    20  60568  []   A   Substitution(type_='SNV', value='C')  NA21144   
15023    20  60571  []   C   Substitution(type_='SNV', value='A')  NA21144   

      genotype  pop super_pop  gender  
0          0|0  GBR    

In [ ]:
import pandas as pd

# Load the CSV file
file_path = '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_CALCIUM_Y2024M06D05.CSV'
calcium_actual = pd.read_csv(file_path)

# Check for missing values
missing_values = calcium_actual.isnull().sum()

# Check for duplicate rows
duplicate_rows = calcium_actual.duplicated().sum()

# Drop unnecessary columns
columns_to_drop = ['measure_id', 'measure_name', 'location_set', 'metric_id', 'metric_name']
calcium_actual_cleaned = calcium_actual.drop(columns=columns_to_drop)

# Display the cleaned DataFrame
print(calcium_actual_cleaned.head())


   location_id location_name  sex_id sex_name  age_group_id age_group_name  \
0            6         China       3     Both            10       25 to 29   
1            6         China       3     Both            11       30 to 34   
2            6         China       3     Both            12       35 to 39   
3            6         China       3     Both            13       40 to 44   
4            6         China       3     Both            14       45 to 49   

   year_id   unit       val     upper     lower  
0     1990  g/day  0.320990  0.337980  0.302906  
1     1990  g/day  0.313640  0.332607  0.295435  
2     1990  g/day  0.327698  0.347042  0.309903  
3     1990  g/day  0.336522  0.356920  0.317888  
4     1990  g/day  0.342058  0.362077  0.322839  


In [ ]:
#Loading each dataset
import pandas as pd

def load_and_clean(file_path, value_column_name):
    """
    Load and clean a CSV file, returning a cleaned DataFrame.
    """
    # Load the dataset
    df = pd.read_csv(file_path)

    # Drop unnecessary columns
    columns_to_drop = ['measure_id', 'measure_name', 'location_set', 'metric_id', 'metric_name',
                       'upper', 'lower', 'unit', 'age_group_id', 'sex_id', 'location_id']
    df = df.drop(columns=columns_to_drop, errors='ignore')

    # Rename the 'val' column to a specific value column name for clarity
    df.rename(columns={'val': value_column_name}, inplace=True)

    # Drop duplicates
    df.drop_duplicates(inplace=True)

    # Standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')

    # Filter for relevant data if needed, for example specific years
    df_filtered = df[(df['year_id'] >= 2000) & (df['year_id'] <= 2020)]

    return df_filtered

# Example usage
# Define the file paths and corresponding column names
import pandas as pd

# Define the file paths and corresponding value column names
file_paths = [
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_CALCIUM_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_FIBER_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_FRUIT_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_LEGUMES_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_MILK_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_NUTS_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_OMEGA3_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_PROCMEAT_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_PUFA_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_REDMEAT_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_SODIUM_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_SSBS_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_TRANSFAT_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_VEG_Y2024M06D05.CSV',
    '/content/drive/MyDrive/AWS_Redshift/IHME_GBD_2021_DIET_RISK_1990_2021/IHME_GBD_2021_DIET_RISK_1990_2021_WHOLEGRAINS_Y2024M06D05.CSV'
]


value_column_names = [
    'calcium_intake_g_per_day',
    'fiber_intake_g_per_day',
    'fruit_intake_g_per_day',
    'legumes_intake_g_per_day',
    'milk_intake_g_per_day',
    'nuts_intake_g_per_day',
    'omega3_intake_g_per_day',
    'processed_meat_intake_g_per_day',
    'pufa_intake_g_per_day',
    'red_meat_intake_g_per_day',
    'sodium_intake_g_per_day',
    'ssbs_intake_g_per_day',  # ssbs: Sugar-Sweetened Beverages
    'transfat_intake_g_per_day',
    'veg_intake_g_per_day',
    'wholegrains_intake_g_per_day',
]

# Load and clean each dataset
cleaned_dataframes = [load_and_clean(file_path, value_column_name)
                      for file_path, value_column_name in zip(file_paths, value_column_names)]

#2.Aggregate and Merge
# Merge all cleaned datasets on common columns such as 'location_name', 'age_group_name', 'year_id'
merged_data = cleaned_dataframes[0]
for df in cleaned_dataframes[1:]:
    merged_data = pd.merge(merged_data, df, on=['location_name', 'age_group_name', 'year_id', 'sex_name'], how='outer')

# Display the merged DataFrame
print(merged_data.head())

# Save the merged DataFrame to a new CSV file
merged_data.to_csv('/content/drive/MyDrive/AWS_Redshift/merged_lifestyle_data.csv', index=False)


  location_name sex_name age_group_name  year_id  calcium_intake_g_per_day  \
0         China     Both       25 to 29     2000                  0.363979   
1         China     Both       30 to 34     2000                  0.351419   
2         China     Both       35 to 39     2000                  0.370753   
3         China     Both       40 to 44     2000                  0.382709   
4         China     Both       45 to 49     2000                  0.389289   

   fiber_intake_g_per_day  fruit_intake_g_per_day  legumes_intake_g_per_day  \
0               13.013235               65.477688                 29.620451   
1               12.361805               62.253029                 30.465697   
2               13.194327               66.207218                 31.423916   
3               13.489024               85.938181                 31.918052   
4               13.940915               84.833026                 32.168174   

   milk_intake_g_per_day  nuts_intake_g_per_day  omega3_

In [ ]:
import pandas as pd

# Load the merged lifestyle data
lifestyle_data_path = '/content/drive/MyDrive/AWS_Redshift/merged_lifestyle_data.csv'
merged_lifestyle_data = pd.read_csv(lifestyle_data_path)

# Map the population codes to full country names
mapped_countries = [country_code_to_name[code] for code in unique_countries]

# Normalize country names in both datasets (lowercase and strip spaces)
merged_lifestyle_data['location_name'] = merged_lifestyle_data['location_name'].str.lower().str.strip()
mapped_countries = [name.lower().strip() for name in mapped_countries]

# Filter the lifestyle data to only include countries present in the genomics data (mapped names)
filtered_lifestyle_data = merged_lifestyle_data[merged_lifestyle_data['location_name'].isin(mapped_countries)]

# Remove rows where sex_name is "Both"
filtered_lifestyle_data = filtered_lifestyle_data[filtered_lifestyle_data['sex_name'] != 'Both']

# Remove the 'age_group_name' column explicitly
filtered_lifestyle_data = filtered_lifestyle_data.drop(columns=['age_group_name'])

# Average values across all other columns (group by location_name, sex_name, and year_id)
filtered_lifestyle_data = filtered_lifestyle_data.groupby(
    ['location_name', 'sex_name', 'year_id'], as_index=False
).mean()

# Save the filtered data to a new CSV file
filtered_lifestyle_data.to_csv('filtered_nutrition_data.csv', index=False)

# Save the filtered data to Google Drive with the name filtered_nutrition_data.csv
filtered_lifestyle_data.to_csv('/content/drive/My Drive/AWS_Redshift/filtered_nutrition_data_v2.csv', index=False)

print(f"Filtered lifestyle data contains {filtered_lifestyle_data.shape[0]} rows.")




Filtered lifestyle data contains 798 rows.


In [ ]:
#Filtering Tobacco data to only include countries present in the genomics dataset
import pandas as pd

# Load the tobacco usage file
tobacco_data_path = '/content/drive/MyDrive/AWS_Redshift/Tobacco_usage_cleaned.xlsx'  # Replace with your actual file path
tobacco_data = pd.read_excel(tobacco_data_path)

# Convert the dictionary values (country names) into a list
genomics_countries_list = list(country_code_to_name.values())

# Filter the tobacco data to only include countries present in genomics data
filtered_tobacco_data = tobacco_data[tobacco_data['location_name'].isin(genomics_countries_list)]

# Save the filtered data to a new CSV file
filtered_tobacco_data.to_csv('filtered_tobacco_data.csv', index=False)
filtered_tobacco_data.to_csv('/content/drive/My Drive/AWS_Redshift/filtered_tobacco_data.csv', index=False)

print(f"Filtered tobacco data contains {filtered_tobacco_data.shape[0]} rows.")


Filtered tobacco data contains 274 rows.


In [ ]:
import pandas as pd

# Load the data
file_path = '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001400.csv'
df = pd.read_csv(file_path)

# Group by 'TimeDim', 'ParentLocation', and 'DisaggregatingDimension1ValueCode', then calculate the mean of 'NumericValue'
df_grouped = df.groupby(['TimeDim', 'ParentLocation', 'DisaggregatingDimension1ValueCode'])['NumericValue'].mean().reset_index()

# Rename columns for clarity
df_grouped.rename(columns={
    'ParentLocation': 'parent_location',
    'TimeDim': 'year',
    'DisaggregatingDimension1ValueCode': 'alcohol_type',
    'NumericValue': 'average_numeric_value'
}, inplace=True)

# Print the grouped and averaged DataFrame
print("\nGrouped and Averaged Data Overview:")
print(df_grouped.head())



Grouped and Averaged Data Overview:
   year parent_location            alcohol_type  average_numeric_value
0  1960        Americas     ALCOHOLTYPE_SA_BEER                   3.75
1  1960        Americas  ALCOHOLTYPE_SA_SPIRITS                   3.26
2  1960        Americas    ALCOHOLTYPE_SA_TOTAL                   7.83
3  1960        Americas     ALCOHOLTYPE_SA_WINE                   0.83
4  1960          Europe     ALCOHOLTYPE_SA_BEER                   4.70


In [ ]:
import pandas as pd

def process_file(file_path):
    df = pd.read_csv(file_path)

    # Filter for years between 2000 and 2020
    df = df[(df['TimeDim'] >= 2000) & (df['TimeDim'] <= 2020)]

    # Group by 'TimeDim', 'ParentLocation', and 'DisaggregatingDimension1ValueCode', then calculate the mean of 'NumericValue'
    df_grouped = df.groupby(['TimeDim', 'ParentLocation', 'DisaggregatingDimension1ValueCode'])['NumericValue'].mean().reset_index()

    # Rename columns for clarity
    df_grouped.rename(columns={
        'ParentLocation': 'parent_location',
        'TimeDim': 'year',
        'DisaggregatingDimension1ValueCode': 'alcohol_type',
        'NumericValue': 'average_numeric_value'
    }, inplace=True)

    return df_grouped

# Example file paths
file_paths = [
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001400.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001404.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001688.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001747.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001751.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001818.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001821.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001823.csv'

]


In [ ]:
def merge_dataframes(file_paths):
    combined_df = pd.DataFrame()  # Initialize an empty DataFrame for combined results

    for file_path in file_paths:
        current_df = process_file(file_path)

        # Merge strategy to add only new countries' data
        if combined_df.empty:
            combined_df = current_df
        else:
            # Merge while ensuring no duplicate entries for countries already included
            combined_df = pd.merge(combined_df, current_df, on=['year', 'parent_location', 'alcohol_type'], how='outer', suffixes=('', '_dup'))

            # Handle duplicates: keep the original if duplicated, otherwise take from the new file
            for col in combined_df.columns:
                if '_dup' in col:
                    combined_df[col.replace('_dup', '')].fillna(combined_df[col], inplace=True)
                    combined_df.drop(columns=col, inplace=True)

    return combined_df

# Process all files and merge
final_data = merge_dataframes(file_paths)
print(final_data.head())
final_data.to_csv('/content/drive/MyDrive/AWS_Redshift/Alc_levels/combined_alcohol_data.csv')


   year parent_location                  alcohol_type  average_numeric_value
0  2000          Africa           ALCOHOLTYPE_SA_BEER               0.983990
1  2000          Africa  ALCOHOLTYPE_SA_OTHER_ALCOHOL               1.515731
2  2000          Africa        ALCOHOLTYPE_SA_SPIRITS               0.545006
3  2000          Africa          ALCOHOLTYPE_SA_TOTAL               3.407420
4  2000          Africa           ALCOHOLTYPE_SA_WINE               0.362693


In [ ]:
file_path = '/content/drive/MyDrive/AWS_Redshift/Alc_levels/mapped_combined_alcohol_data.csv'  # Adjust path if necessary
data = pd.read_csv(file_path)

# Step 3: Explode the 'mapped_country' column into separate rows
import ast

# Convert the 'mapped_country' from string representation to a list
data['mapped_country'] = data['mapped_country'].apply(ast.literal_eval)  # If stored as a string list

# Explode the 'mapped_country' to have one row per country
data_exploded = data.explode('mapped_country')

# Rename 'mapped_country' to 'location_name' and 'year' to 'year_id'
data_exploded.rename(columns={'mapped_country': 'location_name', 'year': 'year_id'}, inplace=True)

# Step 4: Calculate the total value of 'average_numeric_value' per location_name and year_id
# Group by 'location_name' and 'year_id' and sum 'average_numeric_value'
aggregated_data = data_exploded.groupby(['location_name', 'year_id']).agg(
    total_average_value=('average_numeric_value', 'sum')
).reset_index()

# Step 5: Create the 'sex_name' column with 70% for Male and 30% for Female
def split_gender_values(row):
    male_value = row['total_average_value'] * 0.7
    female_value = row['total_average_value'] * 0.3
    return pd.DataFrame({
        'location_name': [row['location_name'], row['location_name']],
        'year_id': [row['year_id'], row['year_id']],
        'sex_name': ['Male', 'Female'],
        'gender_specific_value': [male_value, female_value]
    })

# Apply the gender split function to the aggregated data
gender_split_data = aggregated_data.apply(split_gender_values, axis=1)

# Concatenate the resulting DataFrames
final_data = pd.concat(gender_split_data.tolist(), ignore_index=True)

# Step 6: Save the final result back to Google Drive as a new CSV file
output_file_path = '/content/drive/MyDrive/AWS_Redshift/Alc_levels/mapped_combined_alcohol_data_v2.csv'
final_data.to_csv(output_file_path, index=False)

# Print a message indicating the file was saved successfully
print(f"Final gender-split data has been saved to: {output_file_path}")


Final gender-split data has been saved to: /content/drive/MyDrive/AWS_Redshift/Alc_levels/mapped_combined_alcohol_data_v2.csv


In [ ]:
#CHECKING IF IT'S PROPERLY MERGED
import pandas as pd

def process_file(file_path):
    df = pd.read_csv(file_path)
    df = df[(df['TimeDim'] >= 2000) & (df['TimeDim'] <= 2020)]  # Filter years
    grouped_df = df.groupby(['TimeDim', 'ParentLocation', 'DisaggregatingDimension1ValueCode'])['NumericValue'].mean().reset_index()
    grouped_df.rename(columns={
        'ParentLocation': 'parent_location',
        'TimeDim': 'year',
        'DisaggregatingDimension1ValueCode': 'alcohol_type',
        'NumericValue': 'average_numeric_value'
    }, inplace=True)

    print(f"Processed {file_path}: {len(grouped_df['parent_location'].unique())} countries, {len(grouped_df['year'].unique())} years")
    return grouped_df
def merge_dataframes(file_paths):
    combined_df = pd.DataFrame()

    for file_path in file_paths:
        current_df = process_file(file_path)
        if combined_df.empty:
            combined_df = current_df
        else:
            combined_df = pd.merge(combined_df, current_df, on=['year', 'parent_location', 'alcohol_type'], how='outer', suffixes=('', '_dup'))

            # Resolve any duplicates, prioritizing non-duplicated values
            for col in combined_df.columns:
                if '_dup' in col:
                    combined_df[col.replace('_dup', '')].fillna(combined_df[col], inplace=True)
                    combined_df.drop(columns=col, inplace=True)

        print(f"After merging {file_path}: {combined_df.shape[0]} records")

    return combined_df
file_paths = [
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001400.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001404.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001688.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001747.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001751.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001818.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001821.csv',
    '/content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001823.csv'

]

# Process and merge all files
final_data = merge_dataframes(file_paths)

# Final integrity checks
print("Final merged data overview:")
print(final_data.info())
print("Unique years in final dataset:", final_data['year'].unique())
print("Unique countries in final dataset:", final_data['parent_location'].unique())
print("Any missing values:", final_data.isnull().any())


Processed /content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001400.csv: 6 countries, 21 years
After merging /content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001400.csv: 630 records
Processed /content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001404.csv: 6 countries, 21 years
After merging /content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001404.csv: 1008 records
Processed /content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001688.csv: 6 countries, 21 years
After merging /content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001688.csv: 1008 records
Processed /content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001747.csv: 0 countries, 0 years
After merging /content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001747.csv: 1008 records
Processed /content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001751.csv: 6 countries, 21 years
After merging /content/drive/MyDrive/AWS_Redshift/Alc_levels/SA_0000001751.csv: 1008 records
Processed /content/drive/MyDrive/AWS_Redshift/A

In [ ]:
#parsing through combined_alcohol_data
import pandas as pd

# Load the combined alcohol data
file_path = '/content/drive/MyDrive/AWS_Redshift/Alc_levels/combined_alcohol_data.csv'  # Replace with the actual path to your file
combined_alcohol_data = pd.read_csv(file_path)

# Extract unique country names from the parent_location column
unique_countries = combined_alcohol_data['parent_location'].unique()

# Display the unique country names
print("Unique Country Names in parent_location Column:")
print(unique_countries)

# Optionally, you can save these unique country names to a text file for easier review
with open('unique_country_names.txt', 'w') as f:
    for country in unique_countries:
        f.write(f"{country}\n")

print("Unique country names have been saved to 'unique_country_names.txt'")



Unique Country Names in parent_location Column:
['Africa' 'Americas' 'Eastern Mediterranean' 'Europe' 'South-East Asia'
 'Western Pacific']
Unique country names have been saved to 'unique_country_names.txt'


In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/drive/MyDrive/AWS_Redshift/Tobacco_usage.xlsx'
data = pd.read_excel(file_path)

# Step 1: Remove unnecessary columns
columns_to_remove = ['flag', 'setting_average', 'update', 'dataset_id', 'ci_lb', 'ci_ub', 'population']  # Modify this list as needed
data_cleaned = data.drop(columns=columns_to_remove, errors='ignore')

# Step 2: Rename columns to match your existing dataset structure
data_cleaned.rename(columns={
    'setting': 'location_name',
    'date': 'year_id',
    'subgroup': 'sex_name',
    'estimate': 'smoking_prevalence_percent',  # Example column name
}, inplace=True)

# Step 3: Ensure data types are correct
data_cleaned['year_id'] = pd.to_numeric(data_cleaned['year_id'], errors='coerce')
data_cleaned['smoking_prevalence_percent'] = pd.to_numeric(data_cleaned['smoking_prevalence_percent'], errors='coerce')

# Step 4: Handle missing values
# Fill missing values in numeric columns with 0, or drop these rows if it's more appropriate
data_cleaned['smoking_prevalence_percent'].fillna(0, inplace=True)

# Step 5: Reorder columns to match your existing structure
columns_order = [
    'location_name', 'sex_name', 'year_id', 'smoking_prevalence_percent'
    # Add any other relevant columns that should be included in this order
]
data_cleaned = data_cleaned[columns_order]
averaged_data = data_cleaned.groupby(['location_name', 'year_id', 'sex_name']).agg({
    'smoking_prevalence_percent': 'mean'
}).reset_index()

# Step 6: Save the cleaned and structured data to a new file
Tobacco_usage_cleaned = '/content/drive/MyDrive/AWS_Redshift/Tobacco_usage_cleaned.xlsx'
averaged_data.to_excel(Tobacco_usage_cleaned, index=False)
Tobacco_usage_cleaned_noAVG = '/content/drive/MyDrive/AWS_Redshift/Tobacco_usage_cleaned_noAVG.xlsx'
data_cleaned.to_excel(Tobacco_usage_cleaned_noAVG, index=False)



In [ ]:
#Merging alc_levels with lifestyle data
import pandas as pd

# Assume these are your dataset paths
# Load the processed alcohol data (already merged if you followed the previous steps)
alcohol_data = pd.read_csv('/content/drive/MyDrive/AWS_Redshift/Alc_levels/combined_alcohol_data.csv')

# Load the lifestyle data
lifestyle_data = pd.read_csv('/content/drive/MyDrive/AWS_Redshift/merged_lifestyle_data.csv')

# Print to verify loading
print("Alcohol Data Preview:")
print(alcohol_data.head())
print("\nLifestyle Data Preview:")
print(lifestyle_data.head())
# Rename columns if needed to ensure consistency
alcohol_data.rename(columns={'parent_location': 'country_code'}, inplace=True)
# Ensure lifestyle data has these columns: 'year', 'country_code'


Alcohol Data Preview:
   Unnamed: 0  year parent_location                  alcohol_type  \
0           0  2000          Africa           ALCOHOLTYPE_SA_BEER   
1           1  2000          Africa  ALCOHOLTYPE_SA_OTHER_ALCOHOL   
2           2  2000          Africa        ALCOHOLTYPE_SA_SPIRITS   
3           3  2000          Africa          ALCOHOLTYPE_SA_TOTAL   
4           4  2000          Africa           ALCOHOLTYPE_SA_WINE   

   average_numeric_value  
0               0.983990  
1               1.515731  
2               0.545006  
3               3.407420  
4               0.362692  

Lifestyle Data Preview:
  location_name sex_name age_group_name  year_id  calcium_intake_g_per_day  \
0         China     Both       25 to 29     2000                  0.363979   
1         China     Both       30 to 34     2000                  0.351419   
2         China     Both       35 to 39     2000                  0.370753   
3         China     Both       40 to 44     2000                

In [ ]:
import pandas as pd

# Step 1: Remove the unnamed column from the alcohol data if it exists
alcohol_data = alcohol_data.loc[:, ~alcohol_data.columns.str.contains('^Unnamed')]

# Let's inspect the columns in alcohol_data to ensure the unnamed column is removed
print("Alcohol Data Columns after removing unnamed column:")
print(alcohol_data.columns)

# Step 2: Rename columns in alcohol_data to match those in lifestyle_data for merging
alcohol_data.rename(columns={
    'country_code': 'location_name',  # Rename country_code to location_name for consistency
}, inplace=True)

# Verify the columns again after renaming
print("Updated Alcohol Data Columns after renaming:")
print(alcohol_data.columns)

# Step 3: Pivot the alcohol data so that each alcohol type becomes a separate column
alcohol_pivoted = alcohol_data.pivot_table(
    index=['location_name', 'year'],  # Use the correct column names
    columns='alcohol_type',
    values='average_numeric_value'
).reset_index()

# Flatten the multi-level column index created by pivot_table
alcohol_pivoted.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in alcohol_pivoted.columns]

# Step 4: Merge the pivoted alcohol data with the lifestyle data
merged_data = pd.merge(
    lifestyle_data,   # Lifestyle data
    alcohol_pivoted,  # Alcohol data (pivoted)
    on=['location_name', 'year'],  # Common columns to merge on
    how='outer'  # Use 'outer' to keep all records
)

# Print out the merged DataFrame to inspect
print(merged_data.head())

# Save the merged DataFrame to a CSV file
merged_data.to_csv('/content/drive/MyDrive/AWS_Redshift/merged_lifestyle_data.csv', index=False)




NameError: name 'alcohol_data' is not defined

In [ ]:
#parsing alcohol data

import pandas as pd

# Load the combined alcohol data
file_path = '/content/drive/MyDrive/AWS_Redshift/Alc_levels/combined_alcohol_data.csv'  # Replace with the actual path to your file
combined_alcohol_data = pd.read_csv(file_path)

# Define the mapping from regions to genomic countries
region_to_countries = {
    'Africa': ['Gambia', 'Nigeria', 'Sierra Leone', 'Kenya'],  # ACB, GWD, ESN, MSL, LWK
    'Americas': ['Puerto Rico', 'Colombia', 'Peru', 'Mexico'],  # PUR, CLM, PEL, MXL
    'Eastern Mediterranean': ['Pakistan'],  # PJL
    'Europe': ['United Kingdom', 'Finland', 'Spain', 'Italy'],  # GBR, FIN, IBS, TSI
    'South-East Asia': ['India', 'Bangladesh', 'Sri Lanka'],  # ITU, GIH, BEB, STU
    'Western Pacific': ['China', 'Japan', 'Vietnam']  # CHS, CHB, CDX, JPT, KHV
}

# Reverse the mapping to map from countries back to regions
country_to_region = {country: region for region, countries in region_to_countries.items() for country in countries}

# Function to map region to country based on the genomic countries
def map_region_to_countries(region):
    return region_to_countries.get(region, 'Other')

# Apply the mapping to the parent_location column
combined_alcohol_data['mapped_country'] = combined_alcohol_data['parent_location'].apply(map_region_to_countries)
# Drop the 'Unnamed: 0' column if it exists
if 'Unnamed: 0' in combined_alcohol_data.columns:
    combined_alcohol_data = combined_alcohol_data.drop(columns=['Unnamed: 0'])


# Display the first few rows to verify the mapping
print(combined_alcohol_data.head())


# Optionally, save the mapped data to a new CSV file
mapped_file_path = '/content/drive/MyDrive/AWS_Redshift/Alc_levels/mapped_combined_alcohol_data.csv'
combined_alcohol_data.to_csv(mapped_file_path, index=False)

print(f"The dataset with mapped countries has been saved to {mapped_file_path}")




   year parent_location                  alcohol_type  average_numeric_value  \
0  2000          Africa           ALCOHOLTYPE_SA_BEER               0.983990   
1  2000          Africa  ALCOHOLTYPE_SA_OTHER_ALCOHOL               1.515731   
2  2000          Africa        ALCOHOLTYPE_SA_SPIRITS               0.545006   
3  2000          Africa          ALCOHOLTYPE_SA_TOTAL               3.407420   
4  2000          Africa           ALCOHOLTYPE_SA_WINE               0.362692   

                           mapped_country  
0  [Gambia, Nigeria, Sierra Leone, Kenya]  
1  [Gambia, Nigeria, Sierra Leone, Kenya]  
2  [Gambia, Nigeria, Sierra Leone, Kenya]  
3  [Gambia, Nigeria, Sierra Leone, Kenya]  
4  [Gambia, Nigeria, Sierra Leone, Kenya]  
The dataset with mapped countries has been saved to /content/drive/MyDrive/AWS_Redshift/Alc_levels/mapped_combined_alcohol_data.csv


In [ ]:
#Define the mapping of genomic country codes to full country names
country_code_to_name = {
    'GBR': 'United Kingdom',
    'FIN': 'Finland',
    'CHS': 'China',
    'PUR': 'Puerto Rico',
    'CDX': 'China',
    'CLM': 'Colombia',
    'IBS': 'Spain',
    'PEL': 'Peru',
    'PJL': 'Pakistan',
    'KHV': 'Vietnam',
    'ACB': 'Caribbean',
    'GWD': 'Gambia',
    'ESN': 'Nigeria',
    'BEB': 'Bangladesh',
    'MSL': 'Sierra Leone',
    'STU': 'Sri Lanka',
    'ITU': 'India',
    'CEU': 'United States',
    'YRI': 'Nigeria',
    'CHB': 'China',
    'JPT': 'Japan',
    'LWK': 'Kenya',
    'ASW': 'African American',
    'MXL': 'Mexico',
    'TSI': 'Italy',
    'GIH': 'India'
}

